In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings("ignore")

In [4]:
import sys
sys.path.append("../../") 

from utils.paths import make_dir_line

modality = 'c'
project = 'Introduction to Relational Databases in SQL'
data = make_dir_line(modality, project)

raw = data('raw')

In [5]:
import sqlite3

conn = sqlite3.connect(":memory:")  ## aca se indica el nombre de la db.
cur = conn.cursor()

# 6.4.0_Glue together tables with foreign keys

In [6]:
conn.executescript(
    """
    
    DROP TABLE IF EXISTS university_professors;

    CREATE TABLE university_professors (
        firstname             VARCHAR,
        lastname              VARCHAR,
        university            VARCHAR,
        university_shortname  VARCHAR,
        university_city       VARCHAR,
        function              VARCHAR,
        organization          VARCHAR,
        organization_sector   VARCHAR
    );


    DROP TABLE IF EXISTS professors;

    CREATE TABLE professors (
        firstname            text,
        lastname             text,
        university_shortname text
    );


    DROP TABLE IF EXISTS universities;
    
    CREATE TABLE universities (
        id                   text,
        university           text,
        university_city      text
    );


    DROP TABLE IF EXISTS affiliations;

    CREATE TABLE affiliations (
        firstname             text,
        lastname              text,
        function              text
    );


    DROP TABLE IF EXISTS organizations;

    CREATE TABLE organizations (
        organization        text,
        organization_sector text
    );
    
    
    DROP TABLE IF EXISTS cars;

    CREATE TABLE cars (
        make   varchar(64) NOT NULL,
        model  varchar(64) NOT NULL,
        mpg    integer NOT NULL
        
    );
    

    """
)
conn.commit()

In [7]:
df = pd.read_csv(raw / 'university_professors.csv', sep = ',', decimal = '.', header = 0, encoding = 'utf-8')
university_professors = list(zip(df.firstname, df.lastname, df.university, df.university_shortname, df.university_city, df.function, df.organization, df.organization_sector))
cur.executemany("INSERT INTO university_professors VALUES  (?,?,?,?,?,?,?,?)", university_professors)

df = pd.read_csv(raw / 'cars.csv', sep = ',', decimal = '.', header = 0, encoding = 'utf-8')
cars = list(zip(df.make, df.model, df.mpg))
cur.executemany("INSERT INTO cars VALUES  (?,?,?)", cars)

In [8]:
cur.executescript(
    """
    
    
    INSERT INTO universities 
    SELECT DISTINCT university_shortname, university, university_city 
    FROM university_professors;
    
    
    INSERT INTO professors 
    SELECT DISTINCT firstname, lastname, university_shortname 
    FROM university_professors;


    INSERT INTO affiliations 
    SELECT DISTINCT firstname, lastname, function
    FROM university_professors; 
    
    
    INSERT INTO organizations 
    SELECT DISTINCT organization, organization_sector 
    FROM university_professors; 
    

    """
)
conn.commit()

## 6.4.2 REFERENCE a table with a FOREIGN KEY

In [9]:
# -- Rename the university_shortname column
cur.execute("""
            
            ALTER TABLE professors
            RENAME COLUMN university_shortname TO university_id;

            """).fetchall()

[]

In [10]:
# # -- Add a foreign key on professors referencing universities
# cur.execute("""
            
#             ALTER TABLE professors
#             ADD CONSTRAINT professors_fkey FOREIGN KEY (university_id) REFERENCES universities (id);

#             """).fetchall()

## 6.4.3 Explore foreign key constraints

In [11]:
# -- Try to insert a new professor (fix)
cur.execute("""
            
            INSERT INTO professors (firstname, lastname, university_id)
            VALUES ('Albert', 'Einstein', 'UZH');

            """).fetchall()

[]

## 6.4.4 JOIN tables linked by a foreign key

In [12]:
# -- Select all professors working for universities in the city of Zurich
cur.execute("""
            
            SELECT professors.lastname, universities.id, universities.university_city
            FROM professors
            JOIN universities
            ON professors.university_id = universities.id
            AND universities.university_city = 'Zurich';

            """).fetchall()

[('Abhari', 'ETH', 'Zurich'),
 ('Axhausen', 'ETH', 'Zurich'),
 ('Baschera', 'ETH', 'Zurich'),
 ('Basin', 'ETH', 'Zurich'),
 ('Bechtold', 'ETH', 'Zurich'),
 ('Boutellier', 'ETH', 'Zurich'),
 ('Capkun', 'ETH', 'Zurich'),
 ('Carreira', 'ETH', 'Zurich'),
 ('Chen', 'ETH', 'Zurich'),
 ("D'Andrea", 'ETH', 'Zurich'),
 ('Diederich', 'ETH', 'Zurich'),
 ('Eberle', 'ETH', 'Zurich'),
 ('Ermanni', 'ETH', 'Zurich'),
 ('Faist', 'ETH', 'Zurich'),
 ('Fontana', 'ETH', 'Zurich'),
 ('Frossard', 'ETH', 'Zurich'),
 ('Gramazio', 'ETH', 'Zurich'),
 ('Gross', 'ETH', 'Zurich'),
 ('Grote', 'ETH', 'Zurich'),
 ('Gruber', 'ETH', 'Zurich'),
 ('Gruissem', 'ETH', 'Zurich'),
 ('Guzzella', 'ETH', 'Zurich'),
 ('Günther', 'ETH', 'Zurich'),
 ('Hafen', 'ETH', 'Zurich'),
 ('Heinrich', 'ETH', 'Zurich'),
 ('Hellweg', 'ETH', 'Zurich'),
 ('Holdenrieder', 'ETH', 'Zurich'),
 ('Hungerbühler', 'ETH', 'Zurich'),
 ('Jenny', 'ETH', 'Zurich'),
 ('Kaufmann', 'ETH', 'Zurich'),
 ('Klumpner', 'ETH', 'Zurich'),
 ('Kohler', 'ETH', 'Zurich'),
 

## 6.4.6 Add foreign keys to the "affiliations" table

In [15]:
# -- Add a professor_id column
cur.execute("""
            
            ALTER TABLE affiliations
            ADD COLUMN professor_id integer REFERENCES professors (id);

            """).fetchall()

[]

In [ ]:
# # -- Rename the organization column to organization_id
# cur.execute("""
            
#             ALTER TABLE affiliations
#             RENAME organization TO organization_id

#             """).fetchall()

In [ ]:
# # -- Rename the organization column to organization_id
# cur.execute("""
            
#             ALTER TABLE affiliations
#             ADD CONSTRAINT affiliations_organization_fkey FOREIGN KEY (organization_id) REFERENCES organizations (id);

#             """).fetchall()

## 6.4.7 Populate the "professor_id" column

In [13]:
# -- Have a look at the 10 first rows of affiliations
cur.execute("""
            
            SELECT * 
            FROM affiliations 
            LIMIT 10;

            """).fetchall()

[('Karl', 'Aberer', 'Chairman of L3S Advisory Board'),
 ('Karl', 'Aberer', 'Member Conseil of Zeno-Karl Schindler Foundation'),
 ('Karl', 'Aberer', 'Member of Conseil Fondation IDIAP'),
 ('Karl', 'Aberer', 'Panel Member '),
 ('Reza Shokrollah', 'Abhari', 'Aufsichtsratsmandat'),
 ('Georges',
  'Abou Jaoudé',
  "Professeur invité (2 interventions d'une semaine)"),
 ('Hugues', 'Abriel', None),
 ('Daniel', 'Aebersold', None),
 ('Christoph', 'Aebi', None),
 ('Marcelo', 'Aebi', None)]

In [17]:
# -- Set professor_id to professors.id where firstname, lastname correspond to rows in professors
# cur.execute("""
            
#             UPDATE affiliations
#             SET professor_id = professors.id
#             FROM professors
#             WHERE affiliations.firstname = professors.firstname AND affiliations.lastname = professors.lastname;

#             """).fetchall()

In [18]:
# -- Have a look at the 10 first rows of affiliations again
cur.execute("""
            
            SELECT * 
            FROM affiliations 
            LIMIT 10;

            """).fetchall()

[('Karl', 'Aberer', 'Chairman of L3S Advisory Board', None),
 ('Karl', 'Aberer', 'Member Conseil of Zeno-Karl Schindler Foundation', None),
 ('Karl', 'Aberer', 'Member of Conseil Fondation IDIAP', None),
 ('Karl', 'Aberer', 'Panel Member ', None),
 ('Reza Shokrollah', 'Abhari', 'Aufsichtsratsmandat', None),
 ('Georges',
  'Abou Jaoudé',
  "Professeur invité (2 interventions d'une semaine)",
  None),
 ('Hugues', 'Abriel', None, None),
 ('Daniel', 'Aebersold', None, None),
 ('Christoph', 'Aebi', None, None),
 ('Marcelo', 'Aebi', None, None)]

## 6.4.8 Drop "firstname" and "lastname"

In [19]:
# -- Drop the firstname column
cur.execute("""
            
            ALTER TABLE affiliations
            DROP COLUMN firstname;

            """).fetchall()

[]

In [21]:
# -- Drop the lastname column
cur.execute("""
            
            ALTER TABLE affiliations
            DROP COLUMN lastname;

            """).fetchall()

[]

## 6.4.10 Referential integrity violations

In [ ]:
print('Ok_')

Ok_
